# Ollama 的使用

In [2]:
%%bash

curl -s http://192.168.0.72:11434/v1/chat/completions \
    -H "Content-Type: application/json" \
    -d '{
        "model": "xiaoyu",
        "messages": [
            {
                "role": "user",
                "content": "你是谁"
            }
        ]
    }' | jq .

{
  "id": "chatcmpl-930",
  "object": "chat.completion",
  "created": 1713087035,
  "model": "xiaoyu",
  "system_fingerprint": "fp_ollama",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "我是小羽，由羽胜科技开发的AI助手。我可以回答问题、提供信息和进行对话，帮助你解决各种领域的疑惑。"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 140,
    "completion_tokens": 31,
    "total_tokens": 171
  }
}
